In [2]:
!pip install tensorflow
!pip install matplotlib

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Data loading
(X_data, y_data), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print(X_data.shape)
X_train, y_train = X_data[:50000], y_data[:50000]
X_valid, y_valid = X_data[50000:], y_data[50000:]

del X_data, y_data

print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)

(60000, 28, 28)
(50000, 28, 28) (50000,) (10000, 28, 28) (10000,)


In [4]:
# Dataset standardize
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals) / std_val
X_test_centered = (X_test - mean_vals) / std_val
X_valid_centered = (X_valid - mean_vals) /std_val

del X_train, X_test, X_valid

print(X_train_centered.shape, X_test_centered.shape, X_valid_centered)

(50000, 28, 28) (10000, 28, 28) [[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00 -1.62712835e-05 ... -1.31695701e-04
    0.00000000e+00  0.00000000e+00]
  ...
  [ 0.00000000e+00  0.00000000e+00 -2.38984477e-05 ... -5.44071043e-05
   -2.64408358e-05  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -1.50000895e-05
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00 -1.62712835e-05 ... -1.31695701e-04
    0.00000000e+00  0.00000000e+0

In [5]:
# Data reshaping
X_train_centered = X_train_centered.reshape((-1, 28, 28, 1)) # (28, 28, 1)으로 나눈 후 -1은 나중에 자동으로 한다.
X_test_centered = X_test_centered.reshape((-1, 28, 28, 1))
X_valid_centered = X_valid_centered.reshape((-1, 28, 28, 1))

print(X_train_centered[:2])

# onehot encoding
from tensorflow.keras.utils import to_categorical

y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)
y_valid_onehot = to_categorical(y_valid)

print(y_train_onehot[:3])

[[[[ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   ...
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]]

  [[ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   ...
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]]

  [[ 0.00000000e+00]
   [ 0.00000000e+00]
   [-1.62712835e-05]
   ...
   [-1.31695701e-04]
   [ 0.00000000e+00]
   [ 0.00000000e+00]]

  ...

  [[ 0.00000000e+00]
   [ 0.00000000e+00]
   [-2.38984477e-05]
   ...
   [-5.44071043e-05]
   [-2.64408358e-05]
   [ 0.00000000e+00]]

  [[ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   ...
   [-1.50000895e-05]
   [ 0.00000000e+00]
   [ 0.00000000e+00]]

  [[ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   ...
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]]]


 [[[ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   ...
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]]

  [[ 0.00000000e+00]
   [ 0.00000000e+00]
   [

In [6]:
# Model structure
from tensorflow.keras import layers, models

model = models.Sequential()

model.add(layers.Conv2D(32, (5, 5), padding='valid', activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(64, (5, 5), padding='valid', activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          51264     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 4, 4, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 1024)              1

In [8]:
# Model compiler setting
import time
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
callback_list = [ModelCheckpoint(filepath='cnn_checkpoint.h5', monitor='val_loss', save_best_only=True), TensorBoard(log_dir="logs\{}".format(time.asctime()))]
history = model.fit(X_train_centered, y_train_onehot, batch_size=64, epochs=20, validation_data=(X_valid_centered, y_valid_onehot), callbacks=callback_list)

Epoch 1/20
781/782 [============================>.] - ETA: 0s - loss: 0.1436 - acc: 0.9551

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 119s 147ms/step - loss: 0.1436 - acc: 0.9551 - val_loss: 0.0482 - val_acc: 0.9867
Epoch 2/20
782/782 [==============================] - 114s 146ms/step - loss: 0.0491 - acc: 0.9847 - val_loss: 0.0486 - val_acc: 0.9860
Epoch 3/20
782/782 [==============================] - 117s 149ms/step - loss: 0.0369 - acc: 0.9877 - val_loss: 0.0393 - val_acc: 0.9891
Epoch 4/20
782/782 [==============================] - 114s 145ms/step - loss: 0.0299 - acc: 0.9903 - val_loss: 0.0458 - val_acc: 0.9894
Epoch 5/20
766/782 [============================>.] - ETA: 2s - loss: 0.0234 - acc: 0.9928